In [1]:
import matplotlib
#matplotlib.use('Agg')
%matplotlib tk
%autosave 180
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import numpy as np
import os
from tqdm import trange

from pathlib import Path
from glob import glob


Autosaving every 180 seconds


In [2]:

import os
os.environ["DLClight"] = "True"
import deeplabcut
import tensorflow.contrib.slim as slim
from pathlib import Path
import numpy as np
import tensorflow as tf
vers = (tf.__version__).split(".")
if int(vers[0]) == 1 and int(vers[1]) > 12:
    TF = tf.compat.v1
else:
    TF = tf
    
import tensorflow as tf
from tensorflow.contrib.slim.nets import resnet_v1
from tensorflow.keras.datasets import cifar10
from deeplabcut.pose_estimation_tensorflow.train import  LearningRate



/home/cat/.conda/envs/DLC-GPU/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/cat/.conda/envs/DLC-GPU/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/cat/.conda/envs/DLC-GPU/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/cat/.conda/envs/DLC-GPU/lib/python3.7/site-packages/ten

DLC loaded in light mode; you cannot use any GUI (labeling, relabeling and standalone GUI)


In [3]:
from utility_CNN import (get_optimizer,
                         get_train_config,
                         load_gerbil_training_data,
                         begin_graph,
                         init_iterators,
                         define_loss,
                         extra_info,                         
                         init_session,
                         train_network
)

In [4]:
#
# ------------------- User inputs --------
# define paths
config_path = '/media/cat/4TBSSD/DLC_full_directory/config.yaml'
videofile_path = '/media/cat/4TBSSD/DLC_full_directory/videos_to_label/2020-3-16_12_54_07_193951_compressed_label.mp4'
shuffle = 1 #dlc experiment shuffle

# ------------------- READ DLC info --------
# read config
print("Reading config...")
cfg = deeplabcut.auxiliaryfunctions.read_config(str(config_path))
dlc_cfg = get_train_config(cfg,shuffle=shuffle)

#
# train w imagenet: data/255
BATCH_SIZE = 60
num_classes = 10
is_training=False



Reading config...
/media/cat/4TBSSD/DLC_full_directory/dlc-models/iteration-0/madeline_july2Jul2-trainset95shuffle1/train/pose_cfg.yaml


In [6]:
# LOAD GERBIL DATA SPECIFICALLY FORMATED
#%%
animal_dir ='/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/training_images_cc/segments_training/'
animal_dir = '/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/training_images/animals/'

n_networks = 4

(x_train, y_train, x_test, y_test) = load_gerbil_training_data(animal_dir,
                                                              n_networks)

nx_in, ny_in, nc_in = x_train.shape[1:]
assert nc_in == 3

print ("x train: ", x_train.shape, 
       "y train: ", y_train.shape,  
       "x test: ", x_test.shape, 
       "y test: ", y_test.shape)

(416, 34, 34)
(237, 34, 34)
(660, 34, 34)
(979, 34, 34)
x train:  (2292, 34, 34, 3) y train:  (2292,) x test:  (576, 34, 34, 3) y test:  (576,)


In [7]:
# #%% ------------- BEGIN GRAPH -------------
(image_batch, targets, iterator, train_dataset, test_dataset, batch_size, x, y) = begin_graph(TF,
                                                                            tf,
                                                                            nx_in,
                                                                           ny_in,
                                                                           nc_in)

In [8]:
#     return net_fun
net_fun, inputs, targets, train_init_op, test_init_op = init_iterators(train_dataset,
                              test_dataset,
                              image_batch,
                              resnet_v1,
                              dlc_cfg,
                              iterator,
                              nx_in,
                              tf,
                              targets,
                              batch_size)

(?, 224, 224, 3)


In [9]:
# preprocess the same way as dlc -- here note that they don't normalize
net, logits, loss, tf, logits, loss = define_loss(dlc_cfg,
                               inputs,
                               slim,
                               is_training,
                               tf,
                                resnet_v1,
                                   net_fun,
                                   num_classes,
                                   targets)

In [10]:
# add extrainfo
saver, restorer, variables_to_restore, logits2, classes, accuracy  = extra_info(tf,
                                                  targets,
                                                  logits,
                                                  slim,
                                                  TF)


In [11]:
# %% Init session
dlc_cfg, sess, learning_rate, train_op = init_session(TF,
                                                     loss,
                                                     dlc_cfg,
                                                     restorer,
                                                     tf,
                                                     slim)


LEARNING RATE 1:  Tensor("Placeholder_3:0", shape=(), dtype=float32)
TODO: :USE PRETRAINED SNAPSHOT
Restored variables from
/home/cat/Downloads/resnet_v1_50.ckpt

LEARNING RATE:  Tensor("Placeholder_3:0", shape=(), dtype=float32)


In [12]:
# TRAIN NETWORK
#sess.close()

# train netowrk
if True:
    EPOCHS = 10
    sess = train_network(dlc_cfg, x_train, BATCH_SIZE, y_train,
                      EPOCHS,
                      LearningRate,
                      sess,
                      x_test,
                      y_test,
                      train_init_op,
                      test_init_op,
                      x,
                      y,
                      batch_size,
                      learning_rate,
                      train_op,
                      loss,
                      accuracy,
                      logits2,
                      saver)
    
    model_name = 'snapshots/final'
    saver.save(sess, model_name, global_step=101)

# load previously saved sess
else:
    
    model_name = '/home/cat/code/dlc_toolbox/snapshots/final-101'
    dlc_cfg.init_weights = model_name

    restorer.restore(sess, dlc_cfg.init_weights)
    print('Restored variables from\n{}\n'.format(dlc_cfg.init_weights))



learning_rate:  Tensor("Placeholder_3:0", shape=(), dtype=float32)   current_lr:  0.0001
Training...
Epoch: 0 	 Iter: 0/38, Loss: 30475.7520 Accuracy:0.0000
Epoch: 0 	 Iter: 1/38, Loss: 25084.0664 Accuracy:0.0000
Epoch: 0 	 Iter: 2/38, Loss: 15652.7939 Accuracy:0.0000
Epoch: 0 	 Iter: 3/38, Loss: 2823.4187 Accuracy:0.0000
Epoch: 0 	 Iter: 4/38, Loss: 8615.9785 Accuracy:0.0000
Epoch: 0 	 Iter: 5/38, Loss: 11312.8789 Accuracy:0.0000
Epoch: 0 	 Iter: 6/38, Loss: 4155.6890 Accuracy:0.0000
Epoch: 0 	 Iter: 7/38, Loss: 150.8498 Accuracy:0.8167
Epoch: 0 	 Iter: 8/38, Loss: 9167.5146 Accuracy:0.0000
Epoch: 0 	 Iter: 9/38, Loss: 91406.2031 Accuracy:0.0000
Epoch: 0 	 Iter: 10/38, Loss: 2.8733 Accuracy:0.9667
Epoch: 0 	 Iter: 11/38, Loss: 1222.7469 Accuracy:0.0167
Epoch: 0 	 Iter: 12/38, Loss: 7.8439 Accuracy:0.9667
Epoch: 0 	 Iter: 13/38, Loss: 0.0000 Accuracy:1.0000
Epoch: 0 	 Iter: 14/38, Loss: 0.0000 Accuracy:1.0000
Epoch: 0 	 Iter: 15/38, Loss: 6823.9849 Accuracy:0.0000
Epoch: 0 	 Iter: 16/3

In [16]:
#%% TEST ON THE TRAINING DATA
batch_np = 1
print (x_test.shape)
#for ii ,(image_np, target_np) in enumerate(zip(x_test, y_test)):
ids = np.random.choice(np.arange(x_test.shape[0]), 500, replace=False)
ctr=0
for id_ in ids:
    image_np = x_test[id_][None, :]
    target_np = y_test[id_]
    feed_dict ={image_batch: image_np,
                #targets: target_np,
                batch_size: batch_np}

    classes_np, logits2_np = sess.run([classes, logits2], feed_dict=feed_dict)
    if id_%100==0:
        print ("image: ", id_, "  predicted: ", classes_np[0], " true val: ", target_np, "  confidence: ", logits2_np.max())
        print ('...')
    if classes_np[0]==target_np:
        ctr+=1
    #p#rint('True class {} Predicted class {} Confidence {}'.format(target_np, classes_np[0], logits2_np.max()))
#%%
print ("% Correct: ", ctr/ids.shape[0])


(1468, 34, 34, 3)
image:  100   predicted:  0  true val:  0.0   confidence:  1.0
...
image:  500   predicted:  0  true val:  0.0   confidence:  1.0
...
image:  1100   predicted:  0  true val:  2.0   confidence:  1.0
...
image:  700   predicted:  0  true val:  1.0   confidence:  1.0
...
image:  400   predicted:  0  true val:  0.0   confidence:  1.0
...
% Correct:  0.328


In [1]:
def predict_cnn2(x_test, 
                y_test):
    
    batch_np = 10
    ids = np.arange(0,
                    x_test.shape[0],
                    batch_np)

    ctr=0
    class_array = []
    logit_array = []
    for id_ in ids:

        print ("frame: ", id_)
        image_np = x_test[id_:id_+batch_np] #[None, :]
        target_np = y_test[id_:id_+batch_np]
        feed_dict ={image_batch: image_np,
                    #targets: target_np,
                    batch_size: batch_np}

        classes_np, logits2_np = sess.run([classes, logits2], feed_dict=feed_dict)
        #if classes_np[0]==target_np:
        #    ctr+=1
        class_array.append(classes_np)
        logit_array.append(np.max(logits2))
        
    return ctr, ids.shape[0], class_array, logit_array


def load_gerbil_single_directory(animal_dir,
                                 animal_id):
    
    from skimage.measure import block_reduce
    import glob
    
    data = []
    #print(animal_dir+'/**/*.npy')
    fnames = glob.glob(animal_dir+'*.npy',recursive = True)
    #print ("Fnames; ", len(fnames))
    
    ctr_k = 0
    frame_ids = []
    for ctr, fname in enumerate(fnames):
        if '.npy' in fname:
            temp = np.load(fname)
            idx1 = fname.index('frame_')
            idx2 = fname.index('_id_')
            frame_id = int(fname[idx1+6:idx2])
            #print (frame_id)
            frame_ids.append(frame_id)
            if temp.shape[0]!= 200 or temp.shape[1]!=200:
                continue
            #print (ctr, temp.shape)
            data.append(temp)
            ctr_k+=1
    #print (ctr_k)
            
    data = np.array(data)
    #print ("data: ", 
    #       data.shape)

    x_test = []
    y_test = []
    #data = np.load(animal_dir+"id_"+str(k)+'.npy')
        
    # scale data down:
    #image = np.arange(3*3*4).reshape(3, 3, 4)
    data_downsampled = []
    for p in range(data.shape[0]):
        temp = data[p]
        #print ("temp start: ", temp.shape)
        temp = block_reduce(temp, 
                            block_size=(6,6), 
                            func=np.mean)
        #print ("temp finish: ", temp.shape)
        data_downsampled.append(temp)
    x_test = np.array(data_downsampled)
    #print (x_test.shape)
    y_test = np.zeros(x_test.shape[0])+ animal_id
    
    #x_test = np.vstack(x_test)
    x_test = np.int32((x_test,x_test,x_test)).transpose(1,2,3,0)
    y_test = np.hstack(y_test)[:,None]
        
    #print ("x test: ", x_test.shape,
    #       "y test: ", y_test.shape)
    
    return x_test, y_test, frame_ids

######################################################################
############## ALL DATA FORMATTING FOR CLASSIFICATION ################
######################################################################

# MAKE TRAINING DATA FOR A SINGLE NETWORK
from pathlib import Path
import glob

root_dirs = '/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/training_images_cc/segments_all/'

dirs = glob.glob(root_dirs + '/**')

x_array = []
y_array = []
frames_array = []
vals = []
from tqdm import tqdm

for ctr, dir_ in enumerate(tqdm(dirs)):
    #print (dir_)
    (x_test, y_test, frame_ids) = load_gerbil_single_directory(dir_+'/', ctr)
    #print (x_test.shape, y_test.shape)
    frames_array.append(frame_ids)
    x_array.append(x_test)
    y_array.append(y_test)
    vals.append(np.zeros(x_test.shape[0])+np.int32(os.path.split(dir_)[1]))
    
vals = np.hstack(vals)
#print (vals)
x_test = np.vstack(x_array)
print (x_test.shape)
y_test = np.vstack(y_array)
print (y_test.shape)


    

  0%|          | 0/1264 [00:00<?, ?it/s]


NameError: name 'np' is not defined

In [26]:
print (y_array)

[array([[0.]]), array([[1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.]]), array([[2.],
       [2.],
       [2.],
       [2.],
       [2.],
       [2.],
       [2.],
       [2.],
       [2.],
       [2.],
       [2.],
       [2.],
       [2.],
       [2.],
       [2.],
       [2.],
       [2.],
       [2.],
       [2.],
       [2.]]), array([[3.]]), array([[4.],
       [4.],
       [4.],
       [4.],
       [4.],
       [4.],
       [4.],
       [4.],
       [4.],
       [4.],
       [4.],
       [4.],
       [4.],
       [4.],
       [4.],
       [4.]]), array([[5.],
       [5.]]), array([[6.],
       [6.]]), array([[7.],
       [7.],
       [7.],
       [7.],
       [7.]]), array([[8.],
       [8.],
       [8.],
       [8.],
       [8.],
       [8.],
       [8.],
       [8.]]), array([[9.],
       [9.],
       [9.],
       [9.],
       [9.],
       [9.],
       [9.],
       [9.],
       [9.],
       [9.],
       [9.],
       [9.],
      

In [13]:
# print (np.hstack(frames_array).shape)
# np.save('/home/cat/frames_array_cc.npy',np.hstack(frames_array))

(39225,)


In [14]:
######################################################################
############## RUN CLASSIFIER ON ALL DATA ############################
######################################################################
(ctr, ids, classes, logits) = predict_cnn2(x_test, y_test)
  
classes = np.hstack(classes)
print ("classes: ", classes)

np.savez('/home/cat/testdata.npz',
        x_test=x_test,
        y_test=y_test,
        vals=vals,
        classes=classes,
        frames_array=frames_array)
#print (np.unique(classes, return_counts=True))
    
    

frame:  0
frame:  10
frame:  20
frame:  30
frame:  40
frame:  50
frame:  60
frame:  70
frame:  80
frame:  90
frame:  100
frame:  110
frame:  120
frame:  130
frame:  140
frame:  150
frame:  160
frame:  170
frame:  180
frame:  190
frame:  200
frame:  210
frame:  220
frame:  230
frame:  240
frame:  250
frame:  260
frame:  270
frame:  280
frame:  290
frame:  300
frame:  310
frame:  320
frame:  330
frame:  340
frame:  350
frame:  360
frame:  370
frame:  380
frame:  390
frame:  400
frame:  410
frame:  420
frame:  430
frame:  440
frame:  450
frame:  460
frame:  470
frame:  480
frame:  490
frame:  500
frame:  510
frame:  520
frame:  530
frame:  540
frame:  550
frame:  560
frame:  570
frame:  580
frame:  590
frame:  600
frame:  610
frame:  620
frame:  630
frame:  640
frame:  650
frame:  660
frame:  670
frame:  680
frame:  690
frame:  700
frame:  710
frame:  720
frame:  730
frame:  740
frame:  750
frame:  760
frame:  770
frame:  780
frame:  790
frame:  800
frame:  810
frame:  820
frame:  830
fra

frame:  6390
frame:  6400
frame:  6410
frame:  6420
frame:  6430
frame:  6440
frame:  6450
frame:  6460
frame:  6470
frame:  6480
frame:  6490
frame:  6500
frame:  6510
frame:  6520
frame:  6530
frame:  6540
frame:  6550
frame:  6560
frame:  6570
frame:  6580
frame:  6590
frame:  6600
frame:  6610
frame:  6620
frame:  6630
frame:  6640
frame:  6650
frame:  6660
frame:  6670
frame:  6680
frame:  6690
frame:  6700
frame:  6710
frame:  6720
frame:  6730
frame:  6740
frame:  6750
frame:  6760
frame:  6770
frame:  6780
frame:  6790
frame:  6800
frame:  6810
frame:  6820
frame:  6830
frame:  6840
frame:  6850
frame:  6860
frame:  6870
frame:  6880
frame:  6890
frame:  6900
frame:  6910
frame:  6920
frame:  6930
frame:  6940
frame:  6950
frame:  6960
frame:  6970
frame:  6980
frame:  6990
frame:  7000
frame:  7010
frame:  7020
frame:  7030
frame:  7040
frame:  7050
frame:  7060
frame:  7070
frame:  7080
frame:  7090
frame:  7100
frame:  7110
frame:  7120
frame:  7130
frame:  7140
frame:  7150

frame:  12510
frame:  12520
frame:  12530
frame:  12540
frame:  12550
frame:  12560
frame:  12570
frame:  12580
frame:  12590
frame:  12600
frame:  12610
frame:  12620
frame:  12630
frame:  12640
frame:  12650
frame:  12660
frame:  12670
frame:  12680
frame:  12690
frame:  12700
frame:  12710
frame:  12720
frame:  12730
frame:  12740
frame:  12750
frame:  12760
frame:  12770
frame:  12780
frame:  12790
frame:  12800
frame:  12810
frame:  12820
frame:  12830
frame:  12840
frame:  12850
frame:  12860
frame:  12870
frame:  12880
frame:  12890
frame:  12900
frame:  12910
frame:  12920
frame:  12930
frame:  12940
frame:  12950
frame:  12960
frame:  12970
frame:  12980
frame:  12990
frame:  13000
frame:  13010
frame:  13020
frame:  13030
frame:  13040
frame:  13050
frame:  13060
frame:  13070
frame:  13080
frame:  13090
frame:  13100
frame:  13110
frame:  13120
frame:  13130
frame:  13140
frame:  13150
frame:  13160
frame:  13170
frame:  13180
frame:  13190
frame:  13200
frame:  13210
frame:

frame:  18390
frame:  18400
frame:  18410
frame:  18420
frame:  18430
frame:  18440
frame:  18450
frame:  18460
frame:  18470
frame:  18480
frame:  18490
frame:  18500
frame:  18510
frame:  18520
frame:  18530
frame:  18540
frame:  18550
frame:  18560
frame:  18570
frame:  18580
frame:  18590
frame:  18600
frame:  18610
frame:  18620
frame:  18630
frame:  18640
frame:  18650
frame:  18660
frame:  18670
frame:  18680
frame:  18690
frame:  18700
frame:  18710
frame:  18720
frame:  18730
frame:  18740
frame:  18750
frame:  18760
frame:  18770
frame:  18780
frame:  18790
frame:  18800
frame:  18810
frame:  18820
frame:  18830
frame:  18840
frame:  18850
frame:  18860
frame:  18870
frame:  18880
frame:  18890
frame:  18900
frame:  18910
frame:  18920
frame:  18930
frame:  18940
frame:  18950
frame:  18960
frame:  18970
frame:  18980
frame:  18990
frame:  19000
frame:  19010
frame:  19020
frame:  19030
frame:  19040
frame:  19050
frame:  19060
frame:  19070
frame:  19080
frame:  19090
frame:

frame:  24270
frame:  24280
frame:  24290
frame:  24300
frame:  24310
frame:  24320
frame:  24330
frame:  24340
frame:  24350
frame:  24360
frame:  24370
frame:  24380
frame:  24390
frame:  24400
frame:  24410
frame:  24420
frame:  24430
frame:  24440
frame:  24450
frame:  24460
frame:  24470
frame:  24480
frame:  24490
frame:  24500
frame:  24510
frame:  24520
frame:  24530
frame:  24540
frame:  24550
frame:  24560
frame:  24570
frame:  24580
frame:  24590
frame:  24600
frame:  24610
frame:  24620
frame:  24630
frame:  24640
frame:  24650
frame:  24660
frame:  24670
frame:  24680
frame:  24690
frame:  24700
frame:  24710
frame:  24720
frame:  24730
frame:  24740
frame:  24750
frame:  24760
frame:  24770
frame:  24780
frame:  24790
frame:  24800
frame:  24810
frame:  24820
frame:  24830
frame:  24840
frame:  24850
frame:  24860
frame:  24870
frame:  24880
frame:  24890
frame:  24900
frame:  24910
frame:  24920
frame:  24930
frame:  24940
frame:  24950
frame:  24960
frame:  24970
frame:

frame:  30150
frame:  30160
frame:  30170
frame:  30180
frame:  30190
frame:  30200
frame:  30210
frame:  30220
frame:  30230
frame:  30240
frame:  30250
frame:  30260
frame:  30270
frame:  30280
frame:  30290
frame:  30300
frame:  30310
frame:  30320
frame:  30330
frame:  30340
frame:  30350
frame:  30360
frame:  30370
frame:  30380
frame:  30390
frame:  30400
frame:  30410
frame:  30420
frame:  30430
frame:  30440
frame:  30450
frame:  30460
frame:  30470
frame:  30480
frame:  30490
frame:  30500
frame:  30510
frame:  30520
frame:  30530
frame:  30540
frame:  30550
frame:  30560
frame:  30570
frame:  30580
frame:  30590
frame:  30600
frame:  30610
frame:  30620
frame:  30630
frame:  30640
frame:  30650
frame:  30660
frame:  30670
frame:  30680
frame:  30690
frame:  30700
frame:  30710
frame:  30720
frame:  30730
frame:  30740
frame:  30750
frame:  30760
frame:  30770
frame:  30780
frame:  30790
frame:  30800
frame:  30810
frame:  30820
frame:  30830
frame:  30840
frame:  30850
frame:

frame:  36030
frame:  36040
frame:  36050
frame:  36060
frame:  36070
frame:  36080
frame:  36090
frame:  36100
frame:  36110
frame:  36120
frame:  36130
frame:  36140
frame:  36150
frame:  36160
frame:  36170
frame:  36180
frame:  36190
frame:  36200
frame:  36210
frame:  36220
frame:  36230
frame:  36240
frame:  36250
frame:  36260
frame:  36270
frame:  36280
frame:  36290
frame:  36300
frame:  36310
frame:  36320
frame:  36330
frame:  36340
frame:  36350
frame:  36360
frame:  36370
frame:  36380
frame:  36390
frame:  36400
frame:  36410
frame:  36420
frame:  36430
frame:  36440
frame:  36450
frame:  36460
frame:  36470
frame:  36480
frame:  36490
frame:  36500
frame:  36510
frame:  36520
frame:  36530
frame:  36540
frame:  36550
frame:  36560
frame:  36570
frame:  36580
frame:  36590
frame:  36600
frame:  36610
frame:  36620
frame:  36630
frame:  36640
frame:  36650
frame:  36660
frame:  36670
frame:  36680
frame:  36690
frame:  36700
frame:  36710
frame:  36720
frame:  36730
frame:

In [19]:
######################################################################
############## SHOW EXAMPLES OF CLASSIFIED DATA ######################
######################################################################
start = 0
end = 24
vals_temp = vals[start:end]

data = np.load('/home/cat/testdata.npz')

print (np.unique(vals_temp).shape)
for ctr, dir_ in enumerate(np.unique(vals_temp)):
    idx = np.where(vals_temp==dir_)
    
    temp_classes = classes[idx]
    
    ax=plt.subplot(4,6,ctr+1)
    print (dir_, temp_classes)
plt.show
    
    


(3,)
417.0 [3 3 3 3 3 3 3 3 3 3 3 3 3 3 3]
885.0 [3 0 0 3 3 0 3 3]
1235.0 [3]


/home/cat/.conda/envs/DLC-GPU/lib/python3.7/site-packages/ipykernel_launcher.py:16: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  app.launch_new_instance()


<function matplotlib.pyplot.show(*args, **kw)>

In [ ]:
######################################################################
############## SHOW DISTRIBUTION OF CLASSSES ######################
######################################################################

classes = np.hstack(classes)
print (classes)

y = np.histogram(classes, np.arange(5))
plt.bar(y[1][:-1], y[0], 0.9)
plt.show()

In [1]:
# # 

# def load_gerbil_all_data2(fname_test_data,
#                         animal_id):
    
#     ''' This loads previously saved data arrays from 4 animals
    
#     '''
#     from skimage.measure import block_reduce
    
#     data = np.load(fname_test_data)

#     x_test = []
#     y_test = []
#     #data = np.load(animal_dir+"id_"+str(k)+'.npy')
        
#     # scale data down:
#     #image = np.arange(3*3*4).reshape(3, 3, 4)
#     data_downsampled = []
#     for p in range(data.shape[0]):
#         temp = data[p]
#        # print ("temp start: ", temp.shape)
#         temp = block_reduce(data[p], 
#                             block_size=(6,6), 
#                             func=np.mean)
#         #print ("temp finish: ", temp.shape)
#         data_downsampled.append(temp)
#     x_test = np.array(data_downsampled)
#     #print (x_test.shape)
#     y_test = np.zeros(x_test.shape[0])+ animal_id
    
#     #x_test = np.vstack(x_test)
#     x_test = np.int32((x_test,x_test,x_test)).transpose(1,2,3,0)
#     y_test = np.hstack(y_test)[:,None]
        
#     print ("x test: ", x_test.shape,
#            "y test: ", y_test.shape)
#     return x_test, y_test


# def load_gerbil_all_data(animal_dir,
#                         animal_id):
    
#     from skimage.measure import block_reduce
    
#     names = ['female','male','pup1','pup2']

#     import glob

    
#     if True:
#         data = []
#         fnames = glob.glob(animal_dir+'/**/*',recursive = True)
#         #print ("Fnames; ", fnames)
#         for fname in fnames:
#             if '.npy' in fname:
#                 temp = np.load(fname)
#                 print (temp.shape)
#                 data.append(temp)
                
#         data = np.array(data)
#         print ("data: ", 
#                data.shape)

#     x_test = []
#     y_test = []
#     #data = np.load(animal_dir+"id_"+str(k)+'.npy')
        
#     # scale data down:
#     #image = np.arange(3*3*4).reshape(3, 3, 4)
#     data_downsampled = []
#     for p in range(data.shape[0]):
#         temp = data[p]
#         print ("temp start: ", temp.shape)
#         temp = block_reduce(temp, 
#                             block_size=(6,6), 
#                             func=np.mean)
#         #print ("temp finish: ", temp.shape)
#         data_downsampled.append(temp)
#     x_test = np.array(data_downsampled)
#     print (x_test.shape)
#     y_test = np.zeros(x_test.shape[0])+ animal_id
    
#     #x_test = np.vstack(x_test)
#     x_test = np.int32((x_test,x_test,x_test)).transpose(1,2,3,0)
#     y_test = np.hstack(y_test)[:,None]
        
#     print ("x test: ", x_test.shape,
#            "y test: ", y_test.shape)
#     return x_test, y_test


# animals = ['female','male','pup1','pup2']

# if False:
#     root_dir = '/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/training_images/animals/'

#     for animal_id, animal in enumerate(animals):
#         animal_dir = root_dir + animal

#         (x_test, y_test) = load_gerbil_all_data(animal_dir,
#                             animal_id)

        
#         (ctr, n_images, classes_array, logit_array) = predict_cnn(x_test, 
#                                                                   y_test)

# if False:
#     root_dir = '/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/training_images/animals/id_'
#     for animal_id in range(4):
        
#         fname_test_data = root_dir + str(animal_id) + '_test.npy'
        
#         #if animal_id ==1:
#         #    continue
            
#         (x_test, y_test) =  load_gerbil_all_data2(fname_test_data,
#                         animal_id)
        
#         (ctr, n_images, classes_array, logit_array) = predict_cnn(x_test, 
#                                                                   y_test)
        
#         print (animals[animal_id], animal_id,
#                " # images: ", n_images, 
#                " correct classification: ", ctr/n_images, "%")

In [2]:
# #%% NEW: load one frame
# import tqdm

# def predict_cnn(x_test, 
#                 y_test):
    
#     batch_np = 10
#     ids = np.arange(0,
#                     x_test.shape[0],
#                     batch_np)

#     ctr=0
#     class_array = []
#     logit_array = []
#     for id_ in ids:
#         #if id_%100==0:
#         print ("fraem: ", id_)
#         image_np = x_test[id_:id_+batch_np] #[None, :]
#         target_np = y_test[id_:id_+batch_np]
#         feed_dict ={image_batch: image_np,
#                     #targets: target_np,
#                     batch_size: batch_np}

#         classes_np, logits2_np = sess.run([classes, logits2], feed_dict=feed_dict)
#         #if classes_np[0]==target_np:
#         #    ctr+=1
#         class_array.append(classes_np)
#         logit_array.append(np.max(logits2))
        
#     return ctr, ids.shape[0], class_array, logit_array



In [3]:
# 
# def load_gerbil_single_directory(animal_dir,
#                                  animal_id):
    
#     from skimage.measure import block_reduce
    
#     import glob

    
#     data = []
#     #print(animal_dir+'/**/*.npy')
#     fnames = glob.glob(animal_dir+'/**/*.npy',recursive = True)
#     print ("Fnames; ", len(fnames))
    
#     ctr_k = 0
#     for ctr, fname in enumerate(fnames):
#         if '.npy' in fname:
#             temp = np.load(fname)
#             if temp.shape[0]!= 200 or temp.shape[1]!=200:
#                 continue
#             #print (ctr, temp.shape)
#             data.append(temp)
#             ctr_k+=1
#     print (ctr_k)
            
#     data = np.array(data)
#     print ("data: ", 
#            data.shape)

#     x_test = []
#     y_test = []
#     #data = np.load(animal_dir+"id_"+str(k)+'.npy')
        
#     # scale data down:
#     #image = np.arange(3*3*4).reshape(3, 3, 4)
#     data_downsampled = []
#     for p in range(data.shape[0]):
#         temp = data[p]
#         #print ("temp start: ", temp.shape)
#         temp = block_reduce(temp, 
#                             block_size=(6,6), 
#                             func=np.mean)
#         #print ("temp finish: ", temp.shape)
#         data_downsampled.append(temp)
#     x_test = np.array(data_downsampled)
#     print (x_test.shape)
#     y_test = np.zeros(x_test.shape[0])+ animal_id
    
#     #x_test = np.vstack(x_test)
#     x_test = np.int32((x_test,x_test,x_test)).transpose(1,2,3,0)
#     y_test = np.hstack(y_test)[:,None]
        
#     print ("x test: ", x_test.shape,
#            "y test: ", y_test.shape)
#     return x_test, y_test

# # MAKE TRAINING DATA FOR A SINGLE NETWORK

# # root_dir = '/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/training_images/10000_frames_all_networks/5/'
# # animal_id = 0

# # (x_test, y_test) = load_gerbil_single_directory(root_dir,
# #                                                 animal_id)
# # #print (x_test.shape)

# # print ("DONE Loading data")



In [18]:
# def predict_cnn2(x_test, 
#                 y_test):
    
#     batch_np = 10
#     ids = np.arange(0,
#                     x_test.shape[0],
#                     batch_np)

#     ctr=0
#     class_array = []
#     logit_array = []
#     for id_ in ids:
#         #if id_%100==0:
#         print ("fraem: ", id_)
#         image_np = x_test[id_:id_+batch_np] #[None, :]
#         target_np = y_test[id_:id_+batch_np]
#         feed_dict ={image_batch: image_np,
#                     #targets: target_np,
#                     batch_size: batch_np}

#         classes_np, logits2_np = sess.run([classes, logits2], feed_dict=feed_dict)
#         #if classes_np[0]==target_np:
#         #    ctr+=1
#         class_array.append(classes_np)
#         logit_array.append(np.max(logits2))
        
#     return ctr, ids.shape[0], class_array, logit_array


# root_dir = '/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/training_images/10000_frames_all_networks/'


# current_dir = root_dir
# all_sub_dir_paths = glob(str(current_dir) + '/*/') # returns list of sub directory paths

# all_sub_dir_names = [Path(sub_dir).name for sub_dir in all_sub_dir_paths] 

# print (len(all_sub_dir_names))

# for dir_ in all_sub_dir_names[10:11]:
#     (x_test, y_test) = load_gerbil_single_directory(root_dir+dir_,
#                                                     0)
#     #print (type(x_test))
    
#     (ctr, ids, classes, logits) = predict_cnn2(x_test,
#                                               y_test)
    
#     # 
#     print ("classes: ", classes)
#     print (np.unique(classes, return_counts=True))
#     #break
    

In [38]:
# 

#image_batch = x_test[:10]
#inputs = tf.image.resize_images(image_batch, [224, 224])
#print (inputs.shape)
graph = tf.get_default_graph()
    
feed_dict={x: x_test[:10]}

##model = graph.get_tensor_by_name('resnet_model/final_dense:0')
#model.predict()
model = graph.get_tensor_by_name('/home/cat/code/dlc_toolbox/snapshots/mclass_epoch1-iter32--32.data-00000-of-00001')

#res = sess.run(model, {inputs:img})
    
#res = sess.predict(test_init_op,  
#               feed_dict = feed_dict)




ValueError: The name '/home/cat/code/dlc_toolbox/snapshots/mclass_epoch1-iter32--32.data-00000-of-00001' looks like an (invalid) Operation name, not a Tensor. Tensor names must be of the form "<op_name>:<output_index>".